<a href="https://colab.research.google.com/github/jrmirmi/ML-phenotype-pred/blob/main/Metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IN THIS FILE THERE ARE FUNCTIONS FOR EVALUATION METRICS AND PLOT


In [ ]:
from sklearn.metrics import (
classification_report, 
confusion_matrix, 
accuracy_score, 
balanced_accuracy_score, 
precision_score, recall_score, 
f1_score, roc_curve, 
RocCurveDisplay, 
auc)
import numpy as np
import itertools
import matplotlib.pyplot as plt
import os

In [ ]:
#function for compute the specificity
def specificity_score(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    if cm.shape != (2, 2):
        raise ValueError('The problem must be binary for specificity')
    specificity = cm[0][0] / (cm[0][0] + cm[0][1])
    return specificity

In [ ]:
#function for compute the NPV
def NPV_score(y_true,y_pred):
    cm = confusion_matrix(y_true, y_pred)
    if cm.shape != (2, 2):
        raise ValueError('The problem must be binary for NPV')
    NPV = cm[0][0] / (cm[0][0] + cm[1][0])
    return NPV


In [ ]:
#function for display the ROC curve and the confusion matrix
def plots(cm,fpr,tpr,roc_auc,classes,y_true,y_pred, save_plots):
      # Create a new figure and axes for the confusion matrix and ROC curve plots
    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12, 6))

    # Plot confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    im = ax1.imshow(cm, interpolation='nearest', cmap=plt.cm.RdYlBu)
    ax1.set_title('Confusion matrix')
    ax1.set_xticks(np.arange(len(classes)))
    ax1.set_yticks(np.arange(len(classes)))
    ax1.set_xticklabels(classes, rotation=45)
    ax1.set_yticklabels(classes)
    ax1.set_ylabel('True label')
    ax1.set_xlabel('Predicted label')
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        ax1.text(j, i, format(cm[i, j], '.2f'),
                 horizontalalignment='center',
                 color='black' if cm[i, j] > thresh else 'black')

    # Plot ROC curve
    roc_display = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc, estimator_name='ROC Curve')
    roc_display.plot(ax=ax2)
    ax2.set_title('Receiver Operating Characteristic')
    ax2.set_xlabel('False Positive Rate')
    ax2.set_ylabel('True Positive Rate')
        # Save plot if requested
    if save_plots:
        save_path = os.path.join('.', "roc_cm_plot.png")
        plt.savefig(save_path)

    plt.show()

In [ ]:
def metrics(y_true,y_pred,y_proba,classes, save_plots = False):
    # this is a function that compute the evaluation metrics. the metrics chosen are:
    cm = confusion_matrix(y_true,y_pred)

    #accuracy: function accuracy_score of the sklearn.metrics library
    accuracy = accuracy_score(y_true,y_pred)

    #balanced accuracy: function balanced_accuracy_score of the sklearn.metrics library
    balanced_accuracy = balanced_accuracy_score(y_true,y_pred)

    #precision: function precision_score of the sklearn.metrics library
    precision = precision_score(y_true,y_pred)

    #recall: function recall_score of the sklearn.metrics library
    recall = recall_score(y_true,y_pred, zero_division=1) #zero_division=1 for not true positives

    #specificity: function specificity_score built by us
    specificity = specificity_score(y_true,y_pred)

    #NPV: function negative_predicted_value built by us
    NPV = NPV_score(y_true,y_pred)

    #f1_score: function f1_score of the sklearn.metrics library
    f1 = f1_score(y_true,y_pred)

    #ROC: function roc_curve of the sklearn.metrics library to compute it and the function RocCurveDisplay for the plot
    fpr, tpr, thresholds = roc_curve(y_true, y_proba)

    #AUC: function auc of the sklearn.metrics library
    roc_auc = auc(fpr, tpr)

    plots(cm,fpr,tpr,roc_auc,classes,y_true,y_pred, save_plots)

    # Return evaluation metrics
    return cm, accuracy, balanced_accuracy, precision, recall, specificity, NPV, f1, roc_auc, thresholds




In [ ]:
def metrics_dict(y_true, y_pred, y_proba, classes):
    # Initialize a dictionary to store the metrics for each fold
    metrics_dict = {'cm': [], 'accuracy': [], 'balanced_accuracy': [], 'precision': [], 'recall': [], 'specificity': [], 'NPV': [], 'f1': [], 'roc_auc': [], 'thresholds': []}

    # Compute the evaluation metrics for each fold and store them in the metrics_dict
    for i in range(len(y_true)):
        cm = confusion_matrix(y_true[i], y_pred[i])
        accuracy = accuracy_score(y_true[i], y_pred[i])
        balanced_accuracy = balanced_accuracy_score(y_true[i], y_pred[i])
        precision = precision_score(y_true[i], y_pred[i])
        recall = recall_score(y_true[i], y_pred[i])
        specificity = specificity_score(y_true[i], y_pred[i])
        NPV = NPV_score(y_true[i], y_pred[i])
        f1 = f1_score(y_true[i], y_pred[i])
        fpr, tpr, thresholds = roc_curve(y_true[i], y_proba[i])
        roc_auc = auc(fpr, tpr)

        metrics_dict['cm'].append(cm)
        metrics_dict['accuracy'].append(accuracy)
        metrics_dict['balanced_accuracy'].append(balanced_accuracy)
        metrics_dict['precision'].append(precision)
        metrics_dict['recall'].append(recall)
        metrics_dict['specificity'].append(specificity)
        metrics_dict['NPV'].append(NPV)
        metrics_dict['f1'].append(f1)
        metrics_dict['roc_auc'].append(roc_auc)
        metrics_dict['thresholds'].append(thresholds)

        # Plot the confusion matrix and ROC curve for each fold
        plots(cm, fpr, tpr, roc_auc, classes, y_true[i], y_pred[i])

    # Return the metrics dictionary containing all metrics for each fold
    return metrics_dict
